<a href="https://colab.research.google.com/github/keysenelvial/dinam/blob/main/dinam5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №5**

Провести классификацию найденного датасета, методами решающего дерева и случайного леса.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('/content/drive/MyDrive/train 3.csv', on_bad_lines='skip')
print(data.head())

# Предобработка данных
data = data[['log_price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'property_type']].dropna()
# Разделение цены на категории по квантилям
data['price_category'] = pd.qcut(data['log_price'], q=4, labels=False)

# Уменьшение размера датасета: выбор 15% случайных строк
sampled_data = data.sample(frac=0.15, random_state=42)

# Подготовка признаков и целевой переменной
X = sampled_data[['accommodates', 'bathrooms', 'bedrooms', 'beds']]
y = sampled_data['price_category']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


         id  log_price property_type        room_type  \
0   6901257   5.010635     Apartment  Entire home/apt   
1   6304928   5.129899     Apartment  Entire home/apt   
2   7919400   4.976734     Apartment  Entire home/apt   
3  13418779   6.620073         House  Entire home/apt   
4   3808709   4.744932     Apartment  Entire home/apt   

                                           amenities  accommodates  bathrooms  \
0  {"Wireless Internet","Air conditioning",Kitche...             3        1.0   
1  {"Wireless Internet","Air conditioning",Kitche...             7        1.0   
2  {TV,"Cable TV","Wireless Internet","Air condit...             5        1.0   
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...             4        1.0   
4  {TV,Internet,"Wireless Internet","Air conditio...             2        1.0   

   bed_type cancellation_policy  cleaning_fee  ...   latitude   longitude  \
0  Real Bed              strict          True  ...  40.696524  -73.991617   
1  Real Bed     

In [2]:
# Пайплайны для разных классификаторов
pipe_dt = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

In [3]:
# Параметры для GridSearchCV
params_dt = {
    'clf__max_depth': [None, 10, 20, 30, 40, 50],
    'clf__min_samples_split': [2, 10, 20],
    'clf__min_samples_leaf': [1, 5, 10]
}

params_rf = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 10, 20],
    'clf__min_samples_leaf': [1, 5, 10]
}

# Выбор лучших параметров и обучение моделей
grid_search_dt = GridSearchCV(pipe_dt, param_grid=params_dt, cv=5)
grid_search_rf = GridSearchCV(pipe_rf, param_grid=params_rf, cv=5)

grid_search_dt.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)

print('Лучшие параметры для Decision Tree:', grid_search_dt.best_params_)
print('Лучшие параметры для Random Forest:', grid_search_rf.best_params_)

Лучшие параметры для Decision Tree: {'clf__max_depth': None, 'clf__min_samples_leaf': 10, 'clf__min_samples_split': 2}
Лучшие параметры для Random Forest: {'clf__max_depth': None, 'clf__min_samples_leaf': 10, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}


In [4]:
# Предсказания и отчеты для каждой модели
y_pred_dt = grid_search_dt.predict(X_test)
y_pred_rf = grid_search_rf.predict(X_test)

print("Classification Report for Decision Tree:")
print(classification_report(y_test, y_pred_dt))
print("Classification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))

print("Границы категорий цен:", pd.qcut(data['log_price'], q=4).unique())

Classification Report for Decision Tree:
              precision    recall  f1-score   support

           0       0.39      0.80      0.52       404
           1       0.32      0.05      0.09       459
           2       0.39      0.39      0.39       423
           3       0.66      0.58      0.62       428

    accuracy                           0.45      1714
   macro avg       0.44      0.46      0.41      1714
weighted avg       0.44      0.45      0.40      1714

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.39      0.81      0.53       404
           1       0.31      0.03      0.06       459
           2       0.39      0.39      0.39       423
           3       0.66      0.61      0.64       428

    accuracy                           0.45      1714
   macro avg       0.44      0.46      0.40      1714
weighted avg       0.44      0.45      0.40      1714

Границы категорий цен: [(4.7, 5.165], (4.304, 4.